In [ ]:
from ipynb.fs.full.feature_engineering_final import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_final import TargetNormalizedRegressor
import pandas as pd
from ipynb.fs.full.feature_engineering_final import DataLoader
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
import numpy.ma as ma
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
from mlxtend.regressor import StackingCVRegressor
import seaborn as sns
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNetCV

In [ ]:
def rmsle(y_true, y_pred, sample_weight=None, multioutput='uniform_average'):
    return np.sqrt(mean_squared_log_error(y_true, y_pred, sample_weight, multioutput))
    
#I just want to make rmse positive
flipped_score = make_scorer(rmsle, greater_is_better=True)

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),scoring=None):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
data_loader = DataLoader("train.csv","test.csv")

In [ ]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()

In [ ]:
x_transformer = X_Transformer_scaled()

In [ ]:
X_train = x_transformer.fit_transform(X_train)
kf = KFold(n_splits=10,shuffle = True,random_state = 0)

In [ ]:
#cross validation and predicted vs measured cross validation
xgb = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
svr = SVR(kernel="rbf",gamma=0.0003,C = 30,epsilon = 0.00008)

ridge = RidgeCV(alphas=[14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5])

lasso = LassoCV(cv=10)

elastic_net = ElasticNetCV(max_iter=1e7, alphas=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007],
                                        cv=kf, l1_ratio=[0.8, 0.85, 0.9, 0.95, 0.99, 1])

gbr = GradientBoostingRegressor(n_estimators=3700, learning_rate=0.01,
                                   max_depth=3, max_features='sqrt',
                                   loss='huber', random_state =42,subsample = 0.7)


scvr = TargetNormalizedRegressor(StackingCVRegressor(regressors=(xgb,svr,ridge,lasso,elastic_net,gbr),
                                meta_regressor=elastic_net,cv=10,
                                use_features_in_secondary=True,
                                  n_jobs = -1))





scores = cross_val_score(scvr, X_train, Y_train, cv=kf,n_jobs=-1,scoring = flipped_score)
print("Rmse log average: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))
print("Rmse log median: %0.5f" % (np.median(scores)))
predicted = cross_val_predict(scvr, X_train, Y_train, cv=kf,n_jobs=-1)
print(scores)
fig, ax = plt.subplots()
ax.scatter(Y_train, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
#learning curve of best model
plot_learning_curve(scvr, "Title", X_train, Y_train, None, cv=kf, n_jobs=-1,scoring = flipped_score)